# Основное решение

In [1]:
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor

In [2]:
# Загрузка данных
df = pd.read_csv('C:/develop/Fishing_team/data/dataset.csv')

In [3]:
# Преобразование столбца datetime в datetime формат
df['datetime'] = pd.to_datetime(df['datetime'])

In [4]:
# Создание фичей
df['last_transaction_time'] = df.groupby('client_id')['datetime'].transform('max')
df['time_since_last_transaction'] = (df['last_transaction_time'] - df['datetime']).dt.total_seconds()
pin_attempts = df.groupby('client_id')['pin_inc_count'].sum().reset_index()
pin_attempts = pin_attempts.rename(columns={'pin_inc_count': 'total_pin_incorrect_attempts'})
df = df.merge(pin_attempts, on='client_id', how='left')
unique_ips = df.groupby('client_id')['ip'].nunique().reset_index()
unique_ips = unique_ips.rename(columns={'ip': 'unique_ip_count'})
df = df.merge(unique_ips, on='client_id', how='left')

In [5]:
# Выбираем только числовые фичи для обучения
features = ['time_since_last_transaction', 'total_pin_incorrect_attempts', 'unique_ip_count']
X = df[features]

In [6]:
# Применение Local Outlier Factor (LOF) для обнаружения аномалий
lof = LocalOutlierFactor(n_neighbors=25, contamination=0.005)
df['lof_anomaly'] = lof.fit_predict(X)

In [7]:
# Преобразуем метки из -1 (аномалия) и 1 (норма) в True (аномалия) и False (норма)
df['lof_anomaly'] = df['lof_anomaly'].map({-1: True, 1: False})

In [8]:
# Сохраняем результат в файл без заголовков
df[[ 'lof_anomaly']].to_csv('C:/develop/Fishing_team/data/preds.csv', index=False, header=False)
print("Результат сохранен в 'preds.csv'")

Результат сохранен в 'preds.csv'
